collect tweet

In [3]:
import tweepy
import json
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords
from datetime import datetime
import random

In [6]:
class Story:
    def __init__(self,keywords):
        self.keywords=keywords
    def collect(self):

        consumer_key = "9AZig9wq75kt0FYgjrhQE36LY" 

        consumer_secret = "uHlZ5ymsaOZ3PwDpXzMBpUPTLyVewYSQrgBoygol3BkdTypIyL"

        access_token = "1230732258564132864-yCyObKdgwH8lEWySMzNw9CSXfmcxav"

        access_token_secret = "ssCP3olbTGl3YWjB3zQkz3bvtPRIdiu9nMFuyavWCwgHp"


        auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
        auth.set_access_token(access_token, access_token_secret)
        api = tweepy.API(auth, proxy="127.0.0.1:1081",wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

        #Search specific news account from twitter
        search_results=[]
        #= api.search(lang='en',q='coronavirus',count=100)
        search_results.append(api.user_timeline(id='nytimes',include_rts=False,count=30,tweet_mode="extended"))
        search_results.append(api.user_timeline(id='BreakingNews',include_rts=False,count=30,tweet_mode="extended"))
        search_results.append(api.user_timeline(id='cnnbrk',include_rts=False,count=30,tweet_mode="extended"))
        search_results.append(api.user_timeline(id='WSJbreakingnews',include_rts=False,count=20,tweet_mode="extended"))
        search_results.append(api.user_timeline(id='ABCNewsLive',include_rts=False,count=20,tweet_mode="extended"))
        search_results.append(api.user_timeline(id='SkyNewsBreak',include_rts=False,count=20,tweet_mode="extended"))
        search_results.append(api.user_timeline(id='TWCBreaking',include_rts=False,count=20,tweet_mode="extended"))
        self.pic=[] #get pictures
        
        tweetlist=[]
        for account in search_results:
            for tweet in account:
                if self.keywords in tweet._json['full_text']:
                    tweetlist.append([tweet._json['full_text'].split('http')[0],tweet._json['created_at'].split(' ')[0:3]])
                    media=tweet.entities.get('media', [])
                    if(len(media) > 0):
                          self.pic.append(media[0]['media_url'])
        self.text=[] #get text

        self.timeline=[] #get timeline
        for t in tweetlist:
            txt=t[0].split(':')[-1]

            if len(txt.split())>=10:
                self.text.append(t[0])
                self.timeline.append(t[1])
        return (self.text,self.timeline,self.pic)
    def process(self):
        text1=[]
        for t in self.text:
            t=re.sub(r'[\s]', ' ', t)
            if t[-1:]==' ':
                t=t[:-1]
            if t[-1:]=='.':
                t=t[:-1]
            text1.append(t)
        contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not", "didn't": "did not", "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not",
    "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is", "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would",
    "i'd've": "i would have", "i'll": "i will", "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would",
    "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam",
    "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is",
    "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as",
    "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would",
    "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have",
    "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",
    "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have",
    "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have", "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have",
    "you're": "you are", "you've": "you have"}
        stop_words = set(stopwords.words('english'))
        def text_cleaner(text):
            newString = text.lower()#小写
            newString = BeautifulSoup(newString, "lxml").text
            newString = re.sub('"','', newString)
            newString = ' '.join([contraction_mapping[t] if t in contraction_mapping else t for t in newString.split(" ")])#contraction
            newString = re.sub(r"'s\b","",newString) #去掉’s
            newString = re.sub("[^\.\%0-9a-zA-Z]", " ", newString)
            newString = re.sub('[m]{2,}', 'mm', newString)
            tokens=newString.split()
            return (" ".join(tokens)).strip()

        self.cleaned_text = [] 
        for t in text1:
            self.cleaned_text.append(text_cleaner(t))
    def embedding(self):
        from sklearn.feature_extraction.text import CountVectorizer  

        vectorizer = CountVectorizer()  
 
        X = vectorizer.fit_transform(self.cleaned_text)  
 
        word = vectorizer.get_feature_names()  
        self.text_vec=X.toarray()
    def cluster(self,n_cluster=3):
        from Bio.Cluster import kcluster
        clusterid, error, nfound = kcluster(self.text_vec, n_cluster, dist='u',npass=100)
        self.text_afterclustering={}
        for i in range(0,len(clusterid)):
            if clusterid[i] in self.text_afterclustering:
                self.text_afterclustering[clusterid[i]].append(i)
            else:
                self.text_afterclustering[clusterid[i]]=[i]
                


In [27]:
keywords='coronavirus'

In [7]:
story=Story('coronavirus')
story.collect()
story.process()
story.embedding()
story.cluster()

In [8]:
text_afterclustering=story.text_afterclustering
text=story.text
timeline=story.timeline
pic=story.pic
cleaned_text=story.cleaned_text

In [9]:
text_afterclustering

{0: [0, 4, 5, 8, 9, 10, 12, 13, 15, 18, 23, 27],
 2: [1, 2, 11, 22],
 1: [3, 6, 7, 14, 16, 17, 19, 20, 21, 24, 25, 26]}

In [10]:
pic

['http://pbs.twimg.com/media/EYGluzkWoAE7jcf.jpg',
 'http://pbs.twimg.com/media/EYExmWTXYAAuf5W.jpg',
 'http://pbs.twimg.com/media/EYEUJGxWkAEMXUi.jpg',
 'http://pbs.twimg.com/media/EX_YzVsXQAIk142.jpg',
 'http://pbs.twimg.com/amplify_video_thumb/1261085480902430721/img/CIFrrKHS6nXFgpr7.jpg']

![pic0](http://pbs.twimg.com/media/EYGluzkWoAE7jcf.jpg)

text summarization

In [23]:
#%%we get 2 lists:
import time
from datetime import datetime
times=[]
for i in timeline:
    t='2020 '+' '.join(i)
    t=datetime.strptime(t,'%Y %a %b %d')
    times.append(t)
def sortby_timeline(idlist):
       return (sorted(idlist,key=lambda i:times[i]))

In [24]:
def printstory1(sortedidlist):
    story=''
    for i in sortedidlist:
        story+=cleaned_text[i]+'. '
    return story

In [25]:
def printstory2(sortedidlist):
    story=''
    #先总结
    connectword=['On the same day, ','Also, ',"What's more, ",'While, ','furthermore, ','then, ', 'besides, ']
    former_time=None
    for i in sortedidlist:
        current_time=' '.join(timeline[i][1:3])
        if current_time==former_time:
            n=random.randint(0,6)
            prefix=connectword[n]
        else:
            prefix='On '+current_time+', '
        former_time=current_time
        sen=prefix+cleaned_text[i]+'. '
        story+=sen
    return(story)

In [28]:
storytelling=[]
for i in range(3):
    idlist=text_afterclustering[i]
    sortedidlist=sortby_timeline(idlist)
    smallstory=printstory1(sortedidlist)
    cls=classify(smallstory)
    storytelling.append('Focus on '+cls+' information about '+keywords+'. '+printstory2(sortedidlist))

In [29]:
def wholestory(story):
    ws=''
    for smallstory in story:
        ws+=smallstory+'\n'
    return(ws)
print(wholestory(storytelling))

Focus on life information about coronavirus. On May 08, vp press sec. katie miller tests positive for coronavirus she tells nbcnews saying she is asymptomatic. On May 09, president trump announces the federal government will purchase 3b of dairy meat and produce from farmers as the coronavirus pandemic continues to disrupt the food supply chain. On May 10, uk pm johnson unveils roadmap for getting britain out of the coronavirus lockdown outlining a series of staggered steps that he says would be conditional on how diligently the public follows the government advice. On May 12, live dr. fauci and other top health officials testify at us senate coronavirus hearing. On May 13, india pm modi says 266b will be provided to boost the country economy amid the coronavirus pandemic. furthermore, prime minister boris johnson has paid tribute to railway ticket officer worker belly mujinga who died with coronavirus after being spat at while on duty saying the fact that she was abused for doing her 

# keywords

In [146]:
def idf(corpus):
    idfs = {}
    d = 0.0
    # 统计词出现次数
    for doc in corpus:
        d += 1
        counted = []
        for word in doc.split():
            if not word in counted:
                counted.append(word)
                if word in idfs: 
                    idfs[word] += 1
                else: 
                    idfs[word] = 1
 
    # 计算每个词逆文档值
    for word in idfs:
        idfs[word] = math.log(d/float(idfs[word]))
 
    return idfs

In [147]:
corpus=[]
for i in sortedidlist:
    corpus.append(cleaned_text[i])

In [150]:
import math
idfs = idf(corpus)
for doc in corpus:
    tfidfs = {}
    for word in doc.split():
        if word in tfidfs:    
            tfidfs[word] += 1
        else:
            tfidfs[word] = 1
 #   for word in tfidfs:
  #      tfidfs[word] *= idfs[word]

topic classification

In [11]:
import csv
import pandas as pd
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words('english'))

print(tf.__version__)

2.1.0


In [12]:
vocab_size = 5000
embedding_dim = 64
max_length = 200
trunc_type = 'post'
padding_type = 'post'
oov_tok = '<OOV>'
training_portion = .8

In [13]:
data=pd.read_csv("cat.csv")
articles = []
labels = []
for index, row in data.iterrows():
    labels.append(row[0])
    article = row[1]
    for word in STOPWORDS:
        token = ' ' + word + ' '
        article = article.replace(token, ' ')
        article = article.replace(' ', ' ')
    articles.append(article)

In [14]:
train_size = int(len(articles) * training_portion)

train_articles = articles[0: train_size]
train_labels = labels[0: train_size]

validation_articles = articles[train_size:]
validation_labels = labels[train_size:]

In [15]:
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(train_articles)
word_index = tokenizer.word_index
train_sequences = tokenizer.texts_to_sequences(train_articles)
train_padded = pad_sequences(train_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

In [16]:
validation_sequences = tokenizer.texts_to_sequences(validation_articles)
validation_padded = pad_sequences(validation_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

In [17]:
label_tokenizer = Tokenizer()
label_tokenizer.fit_on_texts(labels)

training_label_seq = np.array(label_tokenizer.texts_to_sequences(train_labels))
validation_label_seq = np.array(label_tokenizer.texts_to_sequences(validation_labels))

In [18]:
index_word=label_tokenizer.index_word

In [19]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_article(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

In [20]:
model = tf.keras.Sequential([
    # Add an Embedding layer expecting input vocab of size 5000, and output embedding dimension of size 64 we set at the top
    tf.keras.layers.Embedding(vocab_size, embedding_dim),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(embedding_dim)),
#    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    # use ReLU in place of tanh function since they are very good alternatives of each other.
    tf.keras.layers.Dense(embedding_dim, activation='relu'),
    # Add a Dense layer with 6 units and softmax activation.
    # When we have multiple outputs, softmax convert outputs layers into a probability distribution.
    tf.keras.layers.Dense(6, activation='softmax')
])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 64)          320000    
_________________________________________________________________
bidirectional (Bidirectional (None, 128)               66048     
_________________________________________________________________
dense (Dense)                (None, 64)                8256      
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 390       
Total params: 394,694
Trainable params: 394,694
Non-trainable params: 0
_________________________________________________________________


In [21]:
import h5py
model.load_weights('classify.h5')

In [22]:
def classify(sentence):
    txt = [sentence]
    seq = tokenizer.texts_to_sequences(txt)
    padded = pad_sequences(seq, maxlen=max_length)
    pred = model.predict(padded)
    label_index_word={1: 'sport', 2: 'business', 3: 'politics', 4: 'tech', 5: 'life'}
    return(label_index_word[np.argmax(pred)])